In [8]:
!pip install cupy-cuda12x


In [12]:
import cupy as cp
import time

N = 1024
A = cp.random.rand(N, N, dtype=cp.float32)
B = cp.random.rand(N, N, dtype=cp.float32)

# warmup
C = A @ B

for block in [(8,8), (16,16), (32,32)]:
    start = time.time()
    C = A @ B   # cuBLAS automatically chooses best block size internally
    cp.cuda.Stream.null.synchronize()
    end = time.time()
    print(f"Block size {block}: {end-start:.4f} seconds")


Block size (8, 8): 0.0012 seconds
Block size (16, 16): 0.0010 seconds
Block size (32, 32): 0.0011 seconds


In [27]:
import cupy as cp
import time
import math

# Matrix size
N = 1024
A = cp.random.rand(N, N, dtype=cp.float32)
B = cp.random.rand(N, N, dtype=cp.float32)
C = cp.zeros((N, N), dtype=cp.float32)

# CUDA Kernel for Matrix Multiplication
matmul_kernel = cp.RawKernel(r'''
extern "C" __global__
void matmul(const float* A, const float* B, float* C, int N) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < N && col < N) {
        float tmp = 0.0f;
        for (int k = 0; k < N; ++k) {
            tmp += A[row * N + k] * B[k * N + col];
        }
        C[row * N + col] = tmp;
    }
}
''', 'matmul')

# Function to run matmul with given block size
def gpu_matmul(A, B, block_size):
    N = A.shape[0]
    C = cp.zeros((N, N), dtype=cp.float32)

    threads_per_block = (block_size, block_size)
    blocks_per_grid_x = math.ceil(N / block_size)
    blocks_per_grid_y = math.ceil(N / block_size)
    blocks_per_grid = (blocks_per_grid_x, blocks_per_grid_y)

    start = time.time()
    matmul_kernel(blocks_per_grid, threads_per_block,
                  (A, B, C, N))
    cp.cuda.Stream.null.synchronize()
    end = time.time()
    return C, end - start

# Test Cases
cases = {
    "Case A (64 threads per block → 8x8)": 8,
    "Case B (256 threads per block → 16x16)": 16,
    "Case C (1024 threads per block → 32x32)": 32,
}

for name, blockdim in cases.items():
    _, t = gpu_matmul(A, B, blockdim)
    print(f"{name}: {t:.4f} seconds")


Case A (64 threads per block → 8x8): 0.0142 seconds
Case B (256 threads per block → 16x16): 0.0090 seconds
Case C (1024 threads per block → 32x32): 0.0066 seconds


In [30]:
import cupy as cp
import time

def best_matmul(A, B, sizes=[64, 256, 1024]):
    best, Cbest = 1e9, None
    for threads in sizes:
        t0 = time.time()
        C = A @ B   # cuBLAS (highly optimized, picks best config itself)
        cp.cuda.Stream.null.synchronize()
        t = time.time() - t0
        print(f"{threads} threads per block (conceptual): {t:.4f}s")
        if t < best:
            best, Cbest = t, C
    return Cbest

# Example usage
N = 1024
A = cp.random.rand(N, N, dtype=cp.float32)
B = cp.random.rand(N, N, dtype=cp.float32)
C = best_matmul(A, B)


64 threads per block (conceptual): 0.0010s
256 threads per block (conceptual): 0.0011s
1024 threads per block (conceptual): 0.0012s
